In [1]:
import pandas as pd

In [2]:
contract = 'MES_stream'

In [3]:
df = pd.read_parquet(f'../futures_ta_data/{contract}_ta_data.parquet')

In [4]:
df.date

0        20200702  18:00:00
1        20200702  18:01:00
2        20200702  18:02:00
3        20200702  18:03:00
4        20200702  18:04:00
                ...        
39623    20200812  22:23:00
39624    20200812  22:24:00
39625    20200812  22:25:00
39626    20200812  22:26:00
39627    20200812  22:27:00
Name: date, Length: 39628, dtype: object

In [4]:
df.loc[:, 'time'] = df.date.apply(lambda x: pd.Timestamp(x))
df.loc[:, 'date'] = df.time.dt.date

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def make_day_plot(day_df):
    seconds = (day_df.time.dt.hour * 3600) + (day_df.time.dt.minute * 60) + (day_df.time.dt.second)
    day_df.loc[:, 'up'] = day_df.open < day_df.close
    fig = make_subplots(rows=3, cols=1, row_heights=[0.5, 0.25, 0.25], shared_xaxes=True)

    fig.add_trace(
        go.Candlestick(
            x=day_df.time, open=day_df.open, high=day_df.high, low=day_df.low, close=day_df.close, name='bars'
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=day_df.time, y=day_df.sma_pr, name='moving average', marker=dict(color='purple')
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=day_df.time, y=day_df.rsi, name='rsi', marker=dict(color='blue')
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=day_df.time, y=day_df.volume, name='volume',
               marker_color=['green' if day_df.loc[index].up else 'red' for index in day_df.index]),
        row=3, col=1
    )
    fig.update_layout(title=f'Contract ', xaxis_rangeslider_visible=False)
    return fig

In [6]:
import plotly

for day in df.date.unique():
    day_df = df.loc[df['date'] == day]
    if day_df.shape[0] > 0:
        fig = make_day_plot(day_df)
        str_date = str(day)
        plotly.offline.plot(fig, filename=f'IntradayViz/{contract}_{str_date}.html', auto_open=False)

/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

